In [ ]:
import os
if os.getcwd().split('\\')[-1] != 'InsertDiffusion' and os.getcwd().split('/')[-1] != 'InsertDiffusion':
    os.chdir('..')

import utils
import importlib
importlib.reload(utils)
%load_ext autoreload
%autoreload 2

import pandas as pd
from PIL import Image

MASK_THRESHOLD = 220
INPAINTING_MODEL = 'stabilityai/stable-diffusion-2-inpainting'
INPAINTING_NEGATIVE_PROMPT = "wrong proportions, outline, white, collage, toy, silhouette, model, clay, above floor, flying, white background, duplicate, multiple, people, distortion, low quality, worst, ugly, fuzzy, blurry, cartoon, simple, art"
INPAINTING_GUIDANCE = 15
INPAINTING_STEPS = 75

IMG2IMG_MODEL = 'stabilityai/stable-diffusion-xl-refiner-1.0'
IMG2IMG_STRENGTH = 0.15
IMG2IMG_GUIDANCE = 7.5

COMPOSITION_STRENGTH = 0.28

Load data

In [ ]:
DATA_PATH = './test_images/tf-icon-bench-ours'
IMAGES_PATH = os.path.join(DATA_PATH, 'fg_images')
BACKGROUNDS_PATH = os.path.join(DATA_PATH, 'bg_images')
INFO_PATH = os.path.join(DATA_PATH, 'info.json')
OUT_PATH = './test_images/eval_results/tf-icon-bench'
os.makedirs(OUT_PATH, exist_ok=True)

In [ ]:
import json

with open(INFO_PATH, 'r') as f:
    info = json.load(f)

Some test

In [ ]:
info['00006']

In [ ]:
test_bg = Image.open(os.path.join(BACKGROUNDS_PATH, '00006.png'))
test_fg = Image.open(os.path.join(IMAGES_PATH, '00006.png'))
test_fg

In [ ]:
test_bg

In [ ]:
test_fg_moved = utils.paste_pipeline(test_fg, info['00006']['scale'], info['00006']['fraction_down'], info['00006']['fraction_right'], rotation=info['00006']['rotation'])
test_fg_moved

In [ ]:
test_cp = utils.paste_image(test_fg_moved, test_bg)
test_cp

In [ ]:
mask = utils.get_mask_from_image(test_fg_moved, MASK_THRESHOLD)
inpainted_test = utils.sd_inpainting(INPAINTING_MODEL, test_cp, mask, info['00006']['prompt'], INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, inpainting_strength=COMPOSITION_STRENGTH)
inpainted_test

In [ ]:
final_test = utils.sd_img2img(IMG2IMG_MODEL, inpainted_test, info['00006']['prompt'], INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE)
final_test

It seems to work on this example (reasonably well, our method performs better with larger objects). Let's do it for all examples.

In [ ]:
examples = sorted([x.split('.')[0] for x in os.listdir(IMAGES_PATH)])

for example in examples:
    c_info = info[example]
    if os.path.exists(os.path.join(IMAGES_PATH, example + '.jpg')):
        c_fg = Image.open(os.path.join(IMAGES_PATH, example + '.jpg'))
    elif os.path.exists(os.path.join(IMAGES_PATH, example + '.png')):
        c_fg = Image.open(os.path.join(IMAGES_PATH, example + '.png'))
    else:
        raise FileNotFoundError(f'Foreground for {example} not found')
    if os.path.exists(os.path.join(BACKGROUNDS_PATH, example + '.jpg')):
        c_bg = Image.open(os.path.join(BACKGROUNDS_PATH, example + '.jpg'))
    elif os.path.exists(os.path.join(BACKGROUNDS_PATH, example + '.png')):
        c_bg = Image.open(os.path.join(BACKGROUNDS_PATH, example + '.png'))
    else:
        raise FileNotFoundError(f'Background for {example} not found')
    
    moved = utils.paste_pipeline(c_fg, c_info['scale'], c_info['fraction_down'], c_info['fraction_right'], rotation=c_info['rotation'])
    mask = utils.get_mask_from_image(moved, MASK_THRESHOLD)

    pasted = utils.paste_image(moved, c_bg)
    
    inpainted = utils.sd_inpainting(INPAINTING_MODEL, pasted, mask, c_info['prompt'], INPAINTING_NEGATIVE_PROMPT, INPAINTING_GUIDANCE, INPAINTING_STEPS, inpainting_strength=COMPOSITION_STRENGTH)
    rediffused = utils.sd_img2img(IMG2IMG_MODEL, inpainted, c_info['prompt'], INPAINTING_NEGATIVE_PROMPT, IMG2IMG_STRENGTH, IMG2IMG_GUIDANCE)

    rediffused.save(os.path.join(OUT_PATH, example + '.png'))